In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
tqdm.pandas()

In [2]:
DATA_PATH = './data/'
DB_PATH = './db/'

yelp_restaurants = pd.read_json(DATA_PATH+'yelp_restaurants.json')
yelp_user = pd.read_json(DATA_PATH+'yelp_user.json')
yelp_review = pd.read_json(DATA_PATH+'yelp_review.json')

In [5]:
# Création des noeuds user

users_csv = yelp_user[["user_id","name","review_count","friends","fans"]]  
users_csv.to_csv(DB_PATH+'users.csv')

# Création des relations friends

friend_relationships_csv = pd.DataFrame(columns = ["user_id","friend_id"])

for i in tqdm(users_csv.index):
    row = users_csv.iloc[i]
    user_id = row['user_id']
    friends = row["friends"]
    relationships = []
    columns = ["user_id","friend_id"]
    for friend_id in friends:
        relationships.append([user_id,friend_id])
    
    df_to_concat = pd.DataFrame(data=relationships,columns=columns)
    friend_relationships_csv = pd.concat([friend_relationships_csv,df_to_concat])

friend_relationships_csv.to_csv(DB_PATH+'friend_relationships.csv')

# Création des noeuds review

reviews_csv = yelp_review[["review_id","stars","useful","cool"]]  
reviews_csv.to_csv(DB_PATH+'reviews.csv')

# Création des relations reviewed

reviewed_relationships_csv = pd.DataFrame(columns=["user_id","review_id"])

for i in tqdm(yelp_review.index):
    row = yelp_review.iloc[i]
    review_id = row["review_id"]
    user_id = row["user_id"]
    df_to_concat = pd.DataFrame(data=[[user_id, review_id]], columns=["user_id", "review_id"])
    
    reviewed_relationships_csv = pd.concat([reviewed_relationships_csv,df_to_concat])

reviewed_relationships_csv.to_csv(DB_PATH+'reviewed_relationships.csv')

100%|███████████████████████████████████████████████████████████████████████████| 49150/49150 [02:44<00:00, 298.87it/s]


In [6]:
friend_relationships_csv = pd.DataFrame(columns = ["user_id","friend_id"])

for i in tqdm(users_csv.index):
    row = users_csv.iloc[i]
    user_id = row['user_id']
    friends = row["friends"]
    relationships = []
    columns = ["user_id","friend_id"]
    for friend_id in friends:
        relationships.append([user_id,friend_id])
    
    df_to_concat = pd.DataFrame(data=relationships,columns=columns)
    friend_relationships_csv = pd.concat([friend_relationships_csv,df_to_concat])

friend_relationships_csv.to_csv(DB_PATH+'friend_relationships.csv')

100%|███████████████████████████████████████████████████████████████████████████| 23082/23082 [01:22<00:00, 279.26it/s]


In [ ]:
restaurants_csv = pd.read_csv(DB_PATH+'restaurants.csv')

for i in tqdm(reviews_csv.index):
    row = reviews_csv.iloc[i]
    review_id = row["review_id"]
    user_id = row["user_id"]
    df_to_concat = pd.DataFrame(data=[[user_id, review_id]], columns=["user_id", "review_id"])
    
    reviewed_relationships_csv = pd.concat([reviewed_relationships_csv,df_to_concat])

reviewed_relationships_csv.to_csv(DB_PATH+'reviewed_relationships.csv')